![Image](./snapshot_strategy.png)

1차에서 수행한 100개의 질문을 제외하고 다시 랜덤하게 100개 추출


In [1]:
file_pre = '4th'

In [2]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
# p = p+r'\config'
sys.path.insert(1, p)
sys.path.insert(1, pp)

In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import config.config as conf
import pickle
import lib.preprocess.preprocess as pp
import lib.preprocess.SectionExtractor as se
import lib.annotation.D_Annotation as da
import lib.annotation.Self_Consistency as sc
import re
import datetime
import pandas as pd
import re
import numpy as np
from sklearn import metrics


INFO 11-26 14:16:45 [__init__.py:216] Automatically detected platform cuda.


In [4]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()
ts = se.SectionExtractor()

In [5]:
q_sql_2223 = """select a.id 
                , a.creationdate
                , a.title
                , b.body
            from posts a 
               , postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2023-12-01' 
              and a.posttypeid = '1'
              and a.tags like '%<python>%' 
              and not exists(select 1 
                                    from public.tt_posts_difficulty_annotated x 
                                where a.id = x.id)
              and not exists(select 1 
                                    from tt_posts_difficulty_done xx 
                                where a.id = xx.id)
"""

In [6]:
# 2324의 경우 public_for_2324 스키마 tt_posts_difficulty_annotated테이블을 보고 제외
q_sql_2324 = """select a.id 
                , a.creationdate
                , a.title
                , b.body
            from public_for_2324.posts a 
               , public_for_2324.postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2024-12-03' 
              and a.posttypeid = '1'
              and a.tags like '%<python>%' 
              and not exists(select 1 
                                    from public_for_2324.tt_posts_difficulty_annotated x 
                                where a.id = x.id)
              and not exists(select 1 
                                    from tt_posts_difficulty_done xx 
                                where a.id = xx.id)
"""

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql_2223)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
q_output_for_2223 = pd.DataFrame(rows, columns = [
  'id'
  ,'creationdate'
  ,'title'
  , 'body'
])


In [ ]:
q_output_for_2223['22_yn'] = np.where(pd.to_datetime(q_output_for_2223['creationdate']).dt.date < datetime.date(2022, 12, 1), 1, 0)

q_output_for_2223_22 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 1, :]
q_output_for_2223_23 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 0, :]
# first_ann_q_id = np.random.choice(list(q_output['id']), size=30, replace=False)

In [ ]:
qid_for_2223_22 = np.random.choice(list(q_output_for_2223_22['id']), size=54, replace=False)
qid_for_2223_23 = np.random.choice(list(q_output_for_2223_23['id']), size=54, replace=False)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql_2324)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
q_output_for_2324 = pd.DataFrame(rows, columns = [
  'id'
  ,'creationdate'
  ,'title'
  , 'body'
])


In [ ]:
q_output_for_2324['22_yn'] = np.where(pd.to_datetime(q_output_for_2324['creationdate']).dt.date < datetime.date(2022, 12, 1), 1, 0)
q_output_for_2324['23_yn'] = np.where((pd.to_datetime(q_output_for_2324['creationdate']).dt.date < datetime.date(2023, 12, 2)) & (q_output_for_2324['22_yn'] ==0), 1, 0)
q_output_for_2324['24_yn'] = np.where(pd.to_datetime(q_output_for_2324['creationdate']).dt.date >= datetime.date(2023, 12, 2), 1, 0)

print(q_output_for_2324.loc[q_output_for_2324['22_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['22_yn'] ==1, 'creationdate'].max())

print(q_output_for_2324.loc[q_output_for_2324['23_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['23_yn'] ==1, 'creationdate'].max())

print(q_output_for_2324.loc[q_output_for_2324['24_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['24_yn'] ==1, 'creationdate'].max())

# q_output_for_2223_22 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 1, :]
# q_output_for_2223_23 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 0, :]
# first_ann_q_id = np.random.choice(list(q_output['id']), size=30, replace=False)

In [ ]:
q_output_for_2324_22 = q_output_for_2324.loc[q_output_for_2324['22_yn'] == 1, :]
q_output_for_2324_23 = q_output_for_2324.loc[q_output_for_2324['23_yn'] == 1, :]
q_output_for_2324_24 = q_output_for_2324.loc[q_output_for_2324['24_yn'] == 1, :]

In [ ]:
print(q_output_for_2324_22['creationdate'].min())
print(q_output_for_2324_22['creationdate'].max())

print(q_output_for_2324_23['creationdate'].min())
print(q_output_for_2324_23['creationdate'].max())

print(q_output_for_2324_24['creationdate'].min())
print(q_output_for_2324_24['creationdate'].max())

In [ ]:
qid_for_2324_22 = np.random.choice(list(q_output_for_2324_22['id']), size=36, replace=False)
qid_for_2324_23 = np.random.choice(list(q_output_for_2324_23['id']), size=36, replace=False)
qid_for_2324_24 = np.random.choice(list(q_output_for_2324_24['id']), size=36, replace=False)

In [ ]:
def chg_tag(code):
    st_pattern = r'<pre(?: class="[^"]*")?><code>'
    st_dst = "```python\n"
    code = re.sub(st_pattern, st_dst, code, count=0, flags=0)
    
    end_dst = "```"
    end_pattern =r'</code></pre>'
    code = re.sub(end_pattern, end_dst, code, count=0, flags=0)
    return code
                                                

In [ ]:
def clean_code_df(q_output, q_list):
    q_output_copy = q_output[q_output['id'].isin(q_list)].copy()
    q_output_copy['t_body'] = q_output_copy['body'].apply(chg_tag)
    q_output_copy['clean_body'] = q_output_copy['t_body'].apply(lambda x : htmlp.get_html_cleaned_str(x))
    q_output_copy['clean_body'] = q_output_copy['clean_body'].apply(lambda x:  re.sub(r";(?=\S)", "", x))
    q_output_copy['question'] = """<Title>"""+q_output_copy['title'].map(str)+"""</Title>. <Question>"""+q_output_copy['clean_body'].map(str)+"""</Question> Let's think through the difficulty of question carefully, step by step. """
    return q_output_copy


In [ ]:
pp_2324_22 = clean_code_df(q_output_for_2223_22, qid_for_2223_22)
pp_2324_22 = clean_code_df(q_output_for_2223_23, qid_for_2223_23)

pp_2324_22 = clean_code_df(q_output_for_2324_22, qid_for_2324_22)
pp_2324_23 = clean_code_df(q_output_for_2324_23, qid_for_2324_23)
pp_2324_24 = clean_code_df(q_output_for_2324_24, qid_for_2324_24)

In [ ]:
snapshop1_sample = pd.concat((pp_2324_22[['id', 'question']], pp_2324_22[['id', 'question']]))
snapshop2_sample = pd.concat((pp_2324_22[['id', 'question']], pp_2324_23[['id', 'question']], pp_2324_24[['id', 'question']]))

In [ ]:
snapshop1_sample[['id', 'question']].to_csv(f'./{file_pre}_snapshop1_sample.csv')
snapshop2_sample[['id', 'question']].to_csv(f'./{file_pre}_snapshop2_sample.csv')

## public_for_2324.tt_posts_difficulty_annotated, public.tt_posts_difficulty_annotated 테이블에 인서트 필요

In [ ]:
list(snapshop1_sample['id'])

In [ ]:
scn_smp = pd.read_csv(f'./2nd_snapshop2_sample.csv')
thd_smp = pd.read_csv(f'./3rd_snapshop2_sample.csv')
tar_smp = pd.read_csv(f'/usr/share/d_ollama/data/q_output_code_y_74_bak.csv')

In [ ]:
sample_smp = pd.concat([scn_smp, thd_smp], axis = 0)

In [ ]:
sample_smp.tail()

In [ ]:
tot_df = pd.merge(tar_smp, sample_smp[['id', 'question']], on = 'id', how='left')

In [ ]:
mapping = {'Basic': 'Difficulty Level : Basic', 
           'Intermediate': 'Difficulty Level : Intermediate', 
           'Advanced' : 'Difficulty Level : Advanced'}
tot_df['answer'] = tot_df['answer'].map(mapping)

In [ ]:
tot_df.to_csv('/usr/share/d_ollama/data/q_output_code_y_74.csv')

In [6]:
from multiprocessing import Process
    
def task(llm_model, few_shot_n, test_n, q_src_yn, ver, p_ver, sc_num, temperature, excel_ver):

    print(f"Task {llm_model}_{few_shot_n}_{test_n}_{q_src_yn}_{p_ver}_{sc_num} 시작")
    for i in range(ver):
        print(f"Task {llm_model}|{few_shot_n}|{test_n}|{q_src_yn}|{ver}|{p_ver}|{sc_num}|{temperature}|{excel_ver} 실행 중: {i}")

        sc.Self_Consistency(   llm_model
                            , few_shot_n
                            , test_n
                            , q_src_yn
                            , ver
                            , p_ver
                            , sc_num
                            , temperature
                            , excel_ver
                            , i)
    
    print(f"Task {llm_model}_{few_shot_n}_{test_n}_{q_src_yn}_{p_ver}_{sc_num} 완료")

if __name__ == "__main__":


    process3 = Process(target=task, args=('v', 4, 5, 'Y', 20, 'sys_prompt10', 5, 0.01, 'ver6'))
    process3.start()
    
    process3.join()
    print("모든 작업 완료")


Task v_4_5_Y_sys_prompt10_5 시작
Task v|4|5|Y|20|sys_prompt10|5|0.01|ver6 실행 중: 0


./result/sc_v_result_4_5_Y_20_sys_prompt10_5_0.01_ver6_0.csv
/usr/share/d_ollama/data/q_output_code_y_74.csv
VLLM
INFO 11-26 14:16:59 [utils.py:328] non-default args: {'tensor_parallel_size': 4, 'gpu_memory_utilization': 0.3, 'disable_log_stats': True, 'model': '/usr/share/d_ollama/.ollama/models/hf_model/Llama-3.2-3B-Instruct'}
INFO 11-26 14:17:06 [__init__.py:742] Resolved architecture: LlamaForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 11-26 14:17:06 [__init__.py:1815] Using max model len 131072


2025-11-26 14:17:07,158	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


...

Task v|4|5|Y|20|sys_prompt10|5|0.01|ver6 실행 중: 19
./result/sc_v_result_4_5_Y_20_sys_prompt10_5_0.01_ver6_19.csv
/usr/share/d_ollama/data/q_output_code_y_74.csv
VLLM
(EngineCore_DP0 pid=2607828) WARNING 11-26 14:31:32 [multiproc_worker_utils.py:273] Reducing Torch parallelism from 32 threads to 1 to avoid unnecessary CPU contention. Set OMP_NUM_THREADS in the external environment to tune this value as needed.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[W1126 14:31:35.921730031 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())
[W1126 14:31:36.441768184 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())
[W1126 14:31:36.591637494 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())
[W1126 14:31:36.593446778 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment var

[Gloo] Rank 0 is connected to 3 peer ranks. Expected number of connected peer ranks is : 3
[Gloo] Rank [Gloo] Rank 1 is connected to 33 is connected to  peer ranks. 3Expected number of connected peer ranks is :  peer ranks. 3Expected number of connected peer ranks is : 3

[Gloo] Rank 2 is connected to 3 peer ranks. Expected number of connected peer ranks is : 3
[Gloo] Rank 0 is connected to 3 peer ranks. Expected number of connected peer ranks is : 3
[Gloo] Rank 1 is connected to 3 peer ranks. Expected number of connected peer ranks is : 3
[Gloo] Rank [Gloo] Rank 3 is connected to 32 peer ranks.  is connected to Expected number of connected peer ranks is : 33 peer ranks. Expected number of connected peer ranks is : 
3
(EngineCore_DP0 pid=2607828) (EngineCore_DP0 pid=2607828) (EngineCore_DP0 pid=2607828) (EngineCore_DP0 pid=2607828) WARNING 11-26 14:31:36 [custom_all_reduce.py:144] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this w

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s];0m 
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  2.02it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  3.13it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  2.88it/s]
(EngineCore_DP0 pid=2607828) (Worker_TP0 pid=2607838) 
Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:05<00:00, 13.34it/s]
Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.82it/s, est. speed input: 10235.78 toks/s, output: 18.50 toks/s]
25it [00:20,  1.24it/s]


Task v_4_5_Y_sys_prompt10_5 완료
모든 작업 완료


In [ ]:
path = './result'
file_list = os.listdir(path)

In [ ]:
def sc_calc_acc_condition_with_temp_with_sc(llm_model, few_shot_n, test_n, q_src_yn, ver, p_ver, sc_num, temp, excel_ver):
    tmp = pd.DataFrame()
    df_eval = pd.DataFrame()
    acc_list = []
    path = './result'
    # ./result/sc_l_result_4_15_Y_30_sys_prompt8_0.01_ver1_0.csv
    file_list = os.listdir(path)
    opt_file = [x for x in file_list if x.startswith(f'sc_{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}_{ver}_{p_ver}_{sc_num}_{temp}_{excel_ver}')]
    opt_file = [x for x in opt_file if x.endswith(f'.csv')]

    df = pd.DataFrame()

    
    if len(opt_file)>0 : 
        for f in opt_file:
            tmp = pd.read_csv(f'{path}/{f}', index_col =0)
            tmp = tmp.dropna()

            tmp['gold'] = tmp['answer_encode'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp['o_result'] = tmp['result'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp = tmp[tmp['o_result'].isin(['1', '0', '2'])]

            
            gold_df = tmp[['id', 'gold']].drop_duplicates()
            chk_cnt = tmp.groupby(['id', 'o_result']).count().reset_index()[['id', 'o_result', 'question']]
            chk_cnt = chk_cnt.rename(columns = {'question': 'cnt'})
            chk_cnt = chk_cnt[chk_cnt['cnt'] == sc_num]
            chk_cnt = chk_cnt.sort_values(by = ['id', 'cnt'], ascending=[True, False]).groupby(['id']).head(1)
            df_eval = pd.merge(gold_df, chk_cnt, on = ['id'])

            df_eval['equal_yn'] = np.where(df_eval['gold']==df_eval['o_result'], 1, 0)
            acc = (df_eval['equal_yn'].sum()/df_eval.shape[0])*100  
            acc_list.append(acc)
            df = pd.concat([df, df_eval], axis =0)
            
        df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
        y_true = df['o_result']
        y_pred = df['gold']
        print(metrics.classification_report(y_true, y_pred, digits=3))
        
        acc = (df['equal_yn'].sum()/df.shape[0])*100            
        print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)
        return acc_list


In [ ]:
# Process(target=task, args=('l', 1, 10, 'Y', 10, 'sys_prompt10', 3, 0.01, 'ver5'))
# Process(target=task, args=('l', 2, 10, 'Y', 10, 'sys_prompt10', 3, 0.01, 'ver5'))
# Process(target=task, args=('l', 4, 10, 'Y', 10, 'sys_prompt10', 5, 0.01, 'ver5'))

In [ ]:
# process3 = Process(target=task, args=                 ('v', 4, 5, 'Y', 1, 'sys_prompt10', 5, 0.01, 'ver6'))
list_ =         sc_calc_acc_condition_with_temp_with_sc('v', 4, 5, 'Y', 1, 'sys_prompt10', 5,  0.01, 'ver6')
print(list_)

### 100개 sample에 대한 난이도 측정 후에 거기서 basic 혹은 advanced에 해당하는것을 뽑아서 sample로 전달하기......

In [ ]:
import lib.annotation.D_Annotation as da
import lib.annotation.Self_Consistency as sc
import lib.annotation.Sample_Insert as si
import lib.annotation.Q_Extract as qe
import lib.annotation.SampleSelf_Consistency as ssc


q_output = pd.read_csv(f'./4th_snapshop2_sample.csv')
    
print(f"SampleSelf_Consistency start")
sample_sc = ssc.SampleSelf_Consistency(q_output) 
print(f"SampleSelf_Consistency end")

print(f"write_promt start")
chk_list = sample_sc.write_promt()
print(f"write_promt end")

print(f"calc_acc_for_v start")
result_df = sample_sc.calc_acc_for_v()
print(f"calc_acc_for_v end")